In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers datasets einops sentencepiece
!pip install scipy scikit-learn

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, platform, warnings
import datasets;
from datasets import load_dataset, Dataset, DatasetDict

from trl import SFTTrainer
from huggingface_hub import notebook_login
#Use a sharded model to fine-tune in the free version of Google Colab.
# base_model = "mistralai/Mistral-7B-v0.1"
base_model = "Trendyol/Trendyol-LLM-7b-chat-dpo-v1.0"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"])
model = get_peft_model(model, peft_config)

In [ ]:
data=data = [
  { 
    "prompts": "Turkcell'in internet hızı çok iyi, fakat Türk Telekom'un müşteri hizmetleri yetersiz.",
    "references": {
      "detect": ["Turkcell'in internet hızı", "Türk Telekom'un müşteri hizmetleri"],
      "sentimentKeywords": {
        "Turkcell'in internet hızı": ["çok iyi"],
        "Türk Telekom'un müşteri hizmetleri": ["yetersiz"]
      }
    }
  },
  {
    "prompts": "Apple'ın yeni iPhone modeli mükemmel, fakat Samsung'un Galaxy serisi daha uygun fiyatlı.",
    "references": {
      "detect": ["Apple'ın yeni iPhone modeli", "Samsung'un Galaxy serisi"],
      "sentimentKeywords": {
        "Apple'ın yeni iPhone modeli": ["mükemmel"],
        "Samsung'un Galaxy serisi": ["uygun fiyatlı"]
      }
    }
  },
  {
    "prompts": "Sony'nin yeni televizyonu inanılmaz bir görüntü kalitesi sunuyor, LG'nin ise ses sistemi zayıf.",
    "references": {
      "detect": ["Sony'nin yeni televizyonu", "LG'nin ses sistemi"],
      "sentimentKeywords": {
        "Sony'nin yeni televizyonu": ["görüntü kalitesi inanılmaz"],
        "LG'nin ses sistemi": ["zayıf"]
      }
    }
  },
  {
    "prompts": "Beko'nun buzdolabı geniş ve kullanışlı, Vestel'in ise enerji verimliliği düşük.",
    "references": {
      "detect": ["Beko'nun buzdolabı", "Vestel'in enerji verimliliği"],
      "sentimentKeywords": {
        "Beko'nun buzdolabı": ["geniş", "kullanışlı"],
        "Vestel'in enerji verimliliği": ["düşük"]
      }
    }
  },
  {
    "prompts": "Xiaomi'nin yeni akıllı saati harika özellikler sunuyor, Huawei'nin ise pil ömrü kısa.",
    "references": {
      "detect": ["Xiaomi'nin yeni akıllı saati", "Huawei'nin pil ömrü"],
      "sentimentKeywords": {
        "Xiaomi'nin yeni akıllı saati": ["harika özellikler"],
        "Huawei'nin pil ömrü": ["kısa"]
      }
    }
  },
  {
    "prompts": "Acer'ın laptopu çok hızlı, fakat Lenovo'nun klavyesi kullanışlı değil.",
    "references": {
      "detect": ["Acer'ın laptopu", "Lenovo'nun klavyesi"],
      "sentimentKeywords": {
        "Acer'ın laptopu": ["çok hızlı"],
        "Lenovo'nun klavyesi": ["kullanışlı değil"]
      }
    }
  },
  {
    "prompts": "Nikon'un yeni DSLR kameraları yüksek çözünürlük sağlıyor, Canon'un ise fiyatı çok yüksek.",
    "references": {
      "detect": ["Nikon'un yeni DSLR kameraları", "Canon'un fiyatı"],
      "sentimentKeywords": {
        "Nikon'un yeni DSLR kameraları": ["yüksek çözünürlük"],
        "Canon'un fiyatı": ["çok yüksek"]
      }
    }
  },
  {
    "prompts": "Microsoft'un Surface tabletleri çok hafif, Samsung'un ise ekran çözünürlüğü düşük.",
    "references": {
      "detect": ["Microsoft'un Surface tabletleri", "Samsung'un ekran çözünürlüğü"],
      "sentimentKeywords": {
        "Microsoft'un Surface tabletleri": ["çok hafif"],
        "Samsung'un ekran çözünürlüğü": ["düşük"]
      }
    }
  },
  {
    "prompts": "Harman Kardon'un ses sistemi gerçekten mükemmel, Bose'un ise bas performansı zayıf.",
    "references": {
      "detect": ["Harman Kardon'un ses sistemi", "Bose'un bas performansı"],
      "sentimentKeywords": {
        "Harman Kardon'un ses sistemi": ["gerçekten mükemmel"],
        "Bose'un bas performansı": ["zayıf"]
      }
    }
  },
  {
    "prompts": "Dell'in monitörü geniş ve keskin bir görüntü sunuyor, Asus'un ise renk doğruluğu düşük.",
    "references": {
      "detect": ["Dell'in monitörü", "Asus'un renk doğruluğu"],
      "sentimentKeywords": {
        "Dell'in monitörü": ["geniş", "keskin görüntü"],
        "Asus'un renk doğruluğu": ["düşük"]
      }
    }
  },
  {
    "prompts": "Google'ın Pixel telefonları çok iyi kamera özellikleri sunuyor, OnePlus'ın ise bataryası kısa ömürlü.",
    "references": {
      "detect": ["Google'ın Pixel telefonları", "OnePlus'ın bataryası"],
      "sentimentKeywords": {
        "Google'ın Pixel telefonları": ["çok iyi kamera özellikleri"],
        "OnePlus'ın bataryası": ["kısa ömürlü"]
      }
    }
  },
  {
    "prompts": "Fujifilm'in fotoğraf makinesi renkleri canlı gösteriyor, Olympus'un ise odaklama hızı yavaş.",
    "references": {
      "detect": ["Fujifilm'in fotoğraf makinesi", "Olympus'un odaklama hızı"],
      "sentimentKeywords": {
        "Fujifilm'in fotoğraf makinesi": ["renkleri canlı gösteriyor"],
        "Olympus'un odaklama hızı": ["yavaş"]
      }
    }
  },
  {
    "prompts": "Huawei'nin telefonları oldukça hızlı, Xiaomi'nin ise ekran çözünürlüğü düşük.",
    "references": {
      "detect": ["Huawei'nin telefonları", "Xiaomi'nin ekran çözünürlüğü"],
      "sentimentKeywords": {
        "Huawei'nin telefonları": ["oldukça hızlı"],
        "Xiaomi'nin ekran çözünürlüğü": ["düşük"]
      }
    }
  },
  {
    "prompts": "Bosch'un bulaşık makinesi sessiz çalışıyor, Arçelik'in ise yıkama performansı düşük.",
    "references": {
      "detect": ["Bosch'un bulaşık makinesi", "Arçelik'in yıkama performansı"],
      "sentimentKeywords": {
        "Bosch'un bulaşık makinesi": ["sessiz çalışıyor"],
        "Arçelik'in yıkama performansı": ["düşük"]
      }
    }
  },
  {
    "prompts": "Sony'nin oyun konsolu grafiklerde mükemmel, Microsoft'un ise kontrol cihazı ergonomik değil.",
    "references": {
      "detect": ["Sony'nin oyun konsolu", "Microsoft'un kontrol cihazı"],
      "sentimentKeywords": {
        "Sony'nin oyun konsolu": ["grafiklerde mükemmel"],
        "Microsoft'un kontrol cihazı": ["ergonomik değil"]
      }
    }
  },
  {
    "prompts": "Razer'ın klavyesi çok hızlı tepki veriyor, Logitech'in ise ışıklandırması zayıf.",
    "references": {
      "detect": ["Razer'ın klavyesi", "Logitech'in ışıklandırması"],
      "sentimentKeywords": {
        "Razer'ın klavyesi": ["çok hızlı tepki veriyor"],
        "Logitech'in ışıklandırması": ["zayıf"]
      }
    }
  },
  {
    "prompts": "Apple'ın AirPods'ları mükemmel ses kalitesi sunuyor, Samsung'un ise bağlantı hızı düşük.",
    "references": {
      "detect": ["Apple'ın AirPods'ları", "Samsung'un bağlantı hızı"],
      "sentimentKeywords": {
        "Apple'ın AirPods'ları": ["mükemmel ses kalitesi"],
        "Samsung'un bağlantı hızı": ["düşük"]
      }
    }
  },
  {
    "prompts": "Philips'in aydınlatma ürünleri uzun ömürlü, GE'nin ise renk sıcaklığı sınırlı.",
    "references": {
      "detect": ["Philips'in aydınlatma ürünleri", "GE'nin renk sıcaklığı"],
      "sentimentKeywords": {
        "Philips'in aydınlatma ürünleri": ["uzun ömürlü"],
        "GE'nin renk sıcaklığı": ["sınırlı"]
      }
    }
  },
    {
    "prompts": "Gopro'nun aksiyon kameraları su geçirmez, DJI'nın ise bataryası çabuk bitiyor.",
    "references": {
      "detect": ["Gopro'nun aksiyon kameraları", "DJI'nın bataryası"],
      "sentimentKeywords": {
        "Gopro'nun aksiyon kameraları": ["su geçirmez"],
        "DJI'nın bataryası": ["çabuk bitiyor"]
      }
    }
  },
  {
    "prompts": "Nespresso'nun kahve makineleri kullanım kolaylığı sağlıyor, Breville'in ise fiyatı yüksek.",
    "references": {
      "detect": ["Nespresso'nun kahve makineleri", "Breville'in fiyatı"],
      "sentimentKeywords": {
        "Nespresso'nun kahve makineleri": ["kullanım kolaylığı"],
        "Breville'in fiyatı": ["yüksek"]
      }
    }
  },
  {
    "prompts": "Corsair'ın RAM'leri yüksek performanslı, Kingston'un ise uyumluluğu sınırlı.",
    "references": {
      "detect": ["Corsair'ın RAM'leri", "Kingston'un uyumluluğu"],
      "sentimentKeywords": {
        "Corsair'ın RAM'leri": ["yüksek performanslı"],
        "Kingston'un uyumluluğu": ["sınırlı"]
      }
    }
  },
  {
    "prompts": "Huawei'nin dizüstü bilgisayarları ince ve hafif, Lenovo'nun ise grafik kartı zayıf.",
    "references": {
      "detect": ["Huawei'nin dizüstü bilgisayarları", "Lenovo'nun grafik kartı"],
      "sentimentKeywords": {
        "Huawei'nin dizüstü bilgisayarları": ["ince", "hafif"],
        "Lenovo'nun grafik kartı": ["zayıf"]
      }
    }
  },
  {
    "prompts": "JBL'in hoparlörleri yüksek ses kalitesi sunuyor, Bose'un ise bas derinliği yeterli değil.",
    "references": {
      "detect": ["JBL'in hoparlörleri", "Bose'un bas derinliği"],
      "sentimentKeywords": {
        "JBL'in hoparlörleri": ["yüksek ses kalitesi"],
        "Bose'un bas derinliği": ["yeterli değil"]
      }
    }
  },
  {
    "prompts": "Dyson'ın süpürgesi güçlü emiş gücüne sahip, Philips'in ise filtre sistemi yetersiz.",
    "references": {
      "detect": ["Dyson'ın süpürgesi", "Philips'in filtre sistemi"],
      "sentimentKeywords": {
        "Dyson'ın süpürgesi": ["güçlü emiş gücü"],
        "Philips'in filtre sistemi": ["yetersiz"]
      }
    }
  },
  {
    "prompts": "Canon'un yeni fotoğraf makinesi renkleri canlı gösteriyor, Nikon'un ise batarya süresi kısa.",
    "references": {
      "detect": ["Canon'un yeni fotoğraf makinesi", "Nikon'un batarya süresi"],
      "sentimentKeywords": {
        "Canon'un yeni fotoğraf makinesi": ["renkleri canlı gösteriyor"],
        "Nikon'un batarya süresi": ["kısa"]
      }
    }
  },
  {
    "prompts": "iPhone 15'in kamerası çok iyi, fakat bataryası hızlı tükeniyor.",
    "references": {
      "detect": ["iPhone 15'in kamerası", "bataryası"],
      "sentimentKeywords": {
        "iPhone 15'in kamerası": ["çok iyi"],
        "bataryası": ["hızlı tükeniyor"]
      }
    }
  },
  {
    "prompts": "Sony WH-1000XM4 kulaklıkları mükemmel ses kalitesi sunuyor, fakat fiyatı oldukça yüksek.",
    "references": {
      "detect": ["Sony WH-1000XM4 kulaklıkları", "fiyatı"],
      "sentimentKeywords": {
        "Sony WH-1000XM4 kulaklıkları": ["mükemmel ses kalitesi"],
        "fiyatı": ["oldukça yüksek"]
      }
    }
  },
        {
    "prompts": "Samsung Galaxy S23'ün ekranı oldukça parlak, ama kameranın gece performansı yeterli değil.",
    "references": {
      "detect": ["Samsung Galaxy S23'ün ekranı", "kameranın gece performansı"],
      "sentimentKeywords": {
        "Samsung Galaxy S23'ün ekranı": ["oldukça parlak"],
        "kameranın gece performansı": ["yeterli değil"]
      }
    }
  },
  {
    "prompts": "Dell XPS 13 bilgisayarı hızlı performans gösteriyor, ancak pil ömrü kısa.",
    "references": {
      "detect": ["Dell XPS 13 bilgisayarı", "pil ömrü"],
      "sentimentKeywords": {
        "Dell XPS 13 bilgisayarı": ["hızlı performans"],
        "pil ömrü": ["kısa"]
      }
    }
  },
  {
    "prompts": "Nikon Z6 fotoğraf makinesi yüksek çözünürlük sağlıyor, ama autofocus sistemi biraz yavaş.",
    "references": {
      "detect": ["Nikon Z6 fotoğraf makinesi", "autofocus sistemi"],
      "sentimentKeywords": {
        "Nikon Z6 fotoğraf makinesi": ["yüksek çözünürlük"],
        "autofocus sistemi": ["biraz yavaş"]
      }
    }
  },
  {
    "prompts": "GoPro Hero11 aksiyon kamerası su geçirmiyor, ancak pil süresi kısıtlı.",
    "references": {
      "detect": ["GoPro Hero11 aksiyon kamerası", "pil süresi"],
      "sentimentKeywords": {
        "GoPro Hero11 aksiyon kamerası": ["su geçirmiyor"],
        "pil süresi": ["kısıtlı"]
      }
    }
  },
  {
    "prompts": "Bose QuietComfort 35 II kulaklıklar mükemmel aktif gürültü engelleme sunuyor, fakat rahatsız edici bir bas var.",
    "references": {
      "detect": ["Bose QuietComfort 35 II kulaklıklar", "rahatsız edici bir bas"],
      "sentimentKeywords": {
        "Bose QuietComfort 35 II kulaklıklar": ["mükemmel aktif gürültü engelleme"],
        "rahatsız edici bir bas": ["rahatsız edici"]
      }
    }
  },
  {
    "prompts": "LG OLED TV'nin renkleri canlı ve keskin, ancak ses kalitesi ortalama.",
    "references": {
      "detect": ["LG OLED TV'nin renkleri", "ses kalitesi"],
      "sentimentKeywords": {
        "LG OLED TV'nin renkleri": ["canlı ve keskin"],
        "ses kalitesi": ["ortalama"]
      }
    }
  },
  {
    "prompts": "Asus ROG Strix laptop mükemmel oyun performansı sunuyor, ama ekranın parlaklığı düşük.",
    "references": {
      "detect": ["Asus ROG Strix laptop", "ekranın parlaklığı"],
      "sentimentKeywords": {
        "Asus ROG Strix laptop": ["mükemmel oyun performansı"],
        "ekranın parlaklığı": ["düşük"]
      }
    }
  },
  {
    "prompts": "Apple Watch Series 8 çok şık bir tasarıma sahip, ancak batarya ömrü beklentileri karşılamıyor.",
    "references": {
      "detect": ["Apple Watch Series 8", "batarya ömrü"],
      "sentimentKeywords": {
        "Apple Watch Series 8": ["çok şık"],
        "batarya ömrü": ["beklentileri karşılamıyor"]
      }
    }
  },
  {
    "prompts": "Xiaomi Mi Band 8'in fitness özellikleri kullanışlı, fakat ekranı yeterince büyük değil.",
    "references": {
      "detect": ["Xiaomi Mi Band 8'in fitness özellikleri", "ekranı"],
      "sentimentKeywords": {
        "Xiaomi Mi Band 8'in fitness özellikleri": ["kullanışlı"],
        "ekranı": ["yeterince büyük değil"]
      }
    }
  },
  {
    "prompts": "Huawei P50 Pro'nun kamerası oldukça etkileyici, ama telefonun tasarımı standart.",
    "references": {
      "detect": ["Huawei P50 Pro'nun kamerası", "telefonun tasarımı"],
      "sentimentKeywords": {
        "Huawei P50 Pro'nun kamerası": ["oldukça etkileyici"],
        "telefonun tasarımı": ["standart"]
      }
    }
  },
  {
    "prompts": "Samsung QLED TV çok iyi renkler sunuyor, fakat ses sistemi zayıf.",
    "references": {
      "detect": ["Samsung QLED TV", "ses sistemi"],
      "sentimentKeywords": {
        "Samsung QLED TV": ["çok iyi renkler"],
        "ses sistemi": ["zayıf"]
      }
    }
  },
  {
    "prompts": "Sony A7 III fotoğraf makinesi profesyonel fotoğrafçılar için ideal, ancak menü sistemi karışık.",
    "references": {
      "detect": ["Sony A7 III fotoğraf makinesi", "menü sistemi"],
      "sentimentKeywords": {
        "Sony A7 III fotoğraf makinesi": ["profesyonel fotoğrafçılar için ideal"],
        "menü sistemi": ["karışık"]
      }
    }
  },
  {
    "prompts": "Microsoft Surface Pro 9 hızlı performans sunuyor, ama ağırlığı biraz fazla.",
    "references": {
      "detect": ["Microsoft Surface Pro 9", "ağırlığı"],
      "sentimentKeywords": {
        "Microsoft Surface Pro 9": ["hızlı performans"],
        "ağırlığı": ["biraz fazla"]
      }
    }
  },
  {
    "prompts": "Garmin Fenix 7 saat oldukça dayanıklı, fakat ekranı güneş ışığında zor okunuyor.",
    "references": {
      "detect": ["Garmin Fenix 7 saat", "ekranı"],
      "sentimentKeywords": {
        "Garmin Fenix 7 saat": ["oldukça dayanıklı"],
        "ekranı": ["güneş ışığında zor okunuyor"]
      }
    }
  },
  {
    "prompts": "Apple MacBook Air 2022 modelinin tasarımı çok ince, ancak işlemci performansı beklentilerimi karşılamıyor.",
    "references": {
      "detect": ["Apple MacBook Air 2022 modelinin tasarımı", "işlemci performansı"],
      "sentimentKeywords": {
        "Apple MacBook Air 2022 modelinin tasarımı": ["çok ince"],
        "işlemci performansı": ["beklentilerimi karşılamıyor"]
      }
    }
  },
  {
    "prompts": "NVIDIA GeForce RTX 4080 ekran kartı mükemmel performans sunuyor, ama fiyatı çok yüksek.",
    "references": {
      "detect": ["NVIDIA GeForce RTX 4080 ekran kartı", "fiyatı"],
      "sentimentKeywords": {
        "NVIDIA GeForce RTX 4080 ekran kartı": ["mükemmel performans"],
        "fiyatı": ["çok yüksek"]
      }
    }
  },
  {
    "prompts": "Jabra Elite 85h kulaklıklar mükemmel batarya ömrü sunuyor, fakat ses kalitesi ortalama.",
    "references": {
      "detect": ["Jabra Elite 85h kulaklıklar", "ses kalitesi"],
      "sentimentKeywords": {
        "Jabra Elite 85h kulaklıklar": ["mükemmel batarya ömrü"],
        "ses kalitesi": ["ortalama"]
      }
    }
  },
  {
    "prompts": "Razer DeathAdder V2 mouse hızlı tepki süresi sunuyor, ama ergonomik değil.",
    "references": {
      "detect": ["Razer DeathAdder V2 mouse", "ergonomik"],
      "sentimentKeywords": {
        "Razer DeathAdder V2 mouse": ["hızlı tepki süresi"],
        "ergonomik": ["değil"]
      }
    }
  },
    {
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Philips Airfryer", "hacmi"],
    "sentimentKeywords": {
      "Philips Airfryer": ["hızlı pişiriyor"],
      "hacmi": ["küçük"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Bosch'un bulaşık makinesi", "kurutma performansı"],
    "sentimentKeywords": {
      "Bosch'un bulaşık makinesi": ["sessiz çalışıyor"],
      "kurutma performansı": ["yetersiz"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Huawei MatePad Pro'nun ekranı", "yazılım desteği"],
    "sentimentKeywords": {
      "Huawei MatePad Pro'nun ekranı": ["çok canlı"],
      "yazılım desteği": ["eksik"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Acer Predator Helios 300 oyun bilgisayarı", "soğutma sistemi"],
    "sentimentKeywords": {
      "Acer Predator Helios 300 oyun bilgisayarı": ["iyi bir performans"],
      "soğutma sistemi": ["yetersiz"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["iPhone 15"],
    "sentimentKeywords": {
      "iPhone 15": ["mükemmel performansı"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Sony WH-1000XM4 kulaklıklar"],
    "sentimentKeywords": {
      "Sony WH-1000XM4 kulaklıklar": ["yüksek ses kalitesi"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Samsung Galaxy S23"],
    "sentimentKeywords": {
      "Samsung Galaxy S23": ["parlak ekranı"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Dell XPS 13"],
    "sentimentKeywords": {
      "Dell XPS 13": ["hızlı performans"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Nikon Z6"],
    "sentimentKeywords": {
      "Nikon Z6": ["yüksek çözünürlük"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["GoPro Hero11"],
    "sentimentKeywords": {
      "GoPro Hero11": ["su geçirmeyen yapısı"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Bose QuietComfort 35 II"],
    "sentimentKeywords": {
      "Bose QuietComfort 35 II": ["mükemmel gürültü engelleme"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["LG OLED TV"],
    "sentimentKeywords": {
      "LG OLED TV": ["canlı renkler"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Asus ROG Strix"],
    "sentimentKeywords": {
      "Asus ROG Strix": ["yüksek oyun performansı"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Apple Watch Series 8"],
    "sentimentKeywords": {
      "Apple Watch Series 8": ["şık tasarım"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Xiaomi Mi Band 8"],
    "sentimentKeywords": {
      "Xiaomi Mi Band 8": ["kullanışlı fitness özellikleri"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Huawei P50 Pro"],
    "sentimentKeywords": {
      "Huawei P50 Pro": ["etkileyici kamera"]
    }
  }
},
{
  "prompts": "Metni inceleyin ve aşağıdaki alanları doldurun:\n\n- Tespit Edilen Terimler: Liste halinde terimleri belirtin.\n- Duygusal Anahtar Kelimeler: Her terim için duygusal anahtar kelimeleri belirtin.",
  "references": {
    "detect": ["Samsung QLED TV"],
    "sentimentKeywords": {
      "Samsung QLED TV": ["çok iyi renkler"]
    }
  }
},{
  "prompts": "Sony A7 III, profesyonel fotoğrafçılar için ideal.",
  "references": {
    "detect": ["Sony A7 III"],
    "sentimentKeywords": {
      "Sony A7 III": ["profesyonel fotoğrafçılar için ideal"]
    }
  }
},
{
  "prompts": "Microsoft Surface Pro 9, hızlı performans sunuyor.",
  "references": {
    "detect": ["Microsoft Surface Pro 9"],
    "sentimentKeywords": {
      "Microsoft Surface Pro 9": ["hızlı performans"]
    }
  }
},
{
  "prompts": "Garmin Fenix 7, dayanıklı bir akıllı saat.",
  "references": {
    "detect": ["Garmin Fenix 7"],
    "sentimentKeywords": {
      "Garmin Fenix 7": ["dayanıklı"]
    }
  }
},
{
  "prompts": "Apple MacBook Air 2022, ince tasarımı ile dikkat çekiyor.",
  "references": {
    "detect": ["Apple MacBook Air 2022"],
    "sentimentKeywords": {
      "Apple MacBook Air 2022": ["ince tasarım"]
    }
  }
},
{
  "prompts": "NVIDIA GeForce RTX 4080, mükemmel performans sunuyor.",
  "references": {
    "detect": ["NVIDIA GeForce RTX 4080"],
    "sentimentKeywords": {
      "NVIDIA GeForce RTX 4080": ["mükemmel performans"]
    }
  }
},
{
  "prompts": "Jabra Elite 85h, uzun batarya ömrü sunuyor.",
  "references": {
    "detect": ["Jabra Elite 85h"],
    "sentimentKeywords": {
      "Jabra Elite 85h": ["uzun batarya ömrü"]
    }
  }
},
{
  "prompts": "Razer DeathAdder V2, hızlı tepki süresine sahip.",
  "references": {
    "detect": ["Razer DeathAdder V2"],
    "sentimentKeywords": {
      "Razer DeathAdder V2": ["hızlı tepki süresi"]
    }
  }
},
{
  "prompts": "Philips Airfryer, hızlı yemek pişirme özelliğine sahip.",
  "references": {
    "detect": ["Philips Airfryer"],
    "sentimentKeywords": {
      "Philips Airfryer": ["hızlı yemek pişirme"]
    }
  }
},
{
  "prompts": "Bosch'un bulaşık makinesi, sessiz çalışıyor.",
  "references": {
    "detect": ["Bosch'un bulaşık makinesi"],
    "sentimentKeywords": {
      "Bosch'un bulaşık makinesi": ["sessiz çalışma"]
    }
  }
},
{
  "prompts": "Bu telefonun kamerası çok iyi, ama bataryası yetersiz.",
  "references": {
    "detect": ["Bu telefonun kamerası", "bataryası"],
    "sentimentKeywords": {
      "Bu telefonun kamerası": ["çok iyi"],
      "bataryası": ["yetersiz"]
    }
  }
},
{
  "prompts": "O kulaklık mükemmel ses kalitesi sunuyor, fakat fiyatı yüksek.",
  "references": {
    "detect": ["O kulaklık", "fiyatı"],
    "sentimentKeywords": {
      "O kulaklık": ["mükemmel ses kalitesi"],
      "fiyatı": ["yüksek"]
    }
  }
},
{
  "prompts": "Şu tabletin ekranı çok net, ancak performansı yeterli değil.",
  "references": {
    "detect": ["Şu tabletin ekranı", "performansı"],
    "sentimentKeywords": {
      "Şu tabletin ekranı": ["çok net"],
      "performansı": ["yeterli değil"]
    }
  }
},
    {
  "prompts": "Onun bilgisayarı hızlı çalışıyor, fakat ekranı küçük.",
  "references": {
    "detect": ["Onun bilgisayarı", "ekranı"],
    "sentimentKeywords": {
      "Onun bilgisayarı": ["hızlı çalışıyor"],
      "ekranı": ["küçük"]
    }
  }
},
{
  "prompts": "Bu saat çok dayanıklı, ama ekranı zor okunuyor.",
  "references": {
    "detect": ["Bu saat", "ekranı"],
    "sentimentKeywords": {
      "Bu saat": ["çok dayanıklı"],
      "ekranı": ["zor okunuyor"]
    }
  }
},
{
  "prompts": "O modelin bataryası uzun ömürlü, ancak tasarımı sıradan.",
  "references": {
    "detect": ["O modelin bataryası", "tasarımı"],
    "sentimentKeywords": {
      "O modelin bataryası": ["uzun ömürlü"],
      "tasarımı": ["sıradan"]
    }
  }
},
{
  "prompts": "Şu cihazın kullanım kolaylığı gerçekten etkileyici, ama şarj süresi kısa.",
  "references": {
    "detect": ["Şu cihazın kullanım kolaylığı", "şarj süresi"],
    "sentimentKeywords": {
      "Şu cihazın kullanım kolaylığı": ["etkileyici"],
      "şarj süresi": ["kısa"]
    }
  }
},
{
  "prompts": "Onların televizyonu harika renkler gösteriyor, ancak ses kalitesi ortalama.",
  "references": {
    "detect": ["Onların televizyonu", "ses kalitesi"],
    "sentimentKeywords": {
      "Onların televizyonu": ["harika renkler gösteriyor"],
      "ses kalitesi": ["ortalama"]
    }
  }
},
{
  "prompts": "Bu bilgisayarın performansı mükemmel, ama ağırlığı fazla.",
  "references": {
    "detect": ["Bu bilgisayarın performansı", "ağırlığı"],
    "sentimentKeywords": {
      "Bu bilgisayarın performansı": ["mükemmel"],
      "ağırlığı": ["fazla"]
    }
  }
},
{
  "prompts": "O kulaklık çok rahat, fakat ses seviyesi yetersiz.",
  "references": {
    "detect": ["O kulaklık", "ses seviyesi"],
    "sentimentKeywords": {
      "O kulaklık": ["çok rahat"],
      "ses seviyesi": ["yetersiz"]
    }
  }
},
{
  "prompts": "Şu cihazın ekranı çok büyük, ama yazılım desteği eksik.",
  "references": {
    "detect": ["Şu cihazın ekranı", "yazılım desteği"],
    "sentimentKeywords": {
      "Şu cihazın ekranı": ["çok büyük"],
      "yazılım desteği": ["eksik"]
    }
  }
},
{
  "prompts": "Bu yeni model mükemmel özelliklere sahip, ancak fiyatı yüksek.",
  "references": {
    "detect": ["Bu yeni model", "fiyatı"],
    "sentimentKeywords": {
      "Bu yeni model": ["mükemmel özelliklere sahip"],
      "fiyatı": ["yüksek"]
    }
  }
},
{
  "prompts": "O telefonda çok iyi bir kamera var, ama ekran kalitesi düşük.",
  "references": {
    "detect": ["O telefonda kamera", "ekran kalitesi"],
    "sentimentKeywords": {
      "O telefonda kamera": ["çok iyi"],
      "ekran kalitesi": ["düşük"]
    }
  }
},
{
  "prompts": "Bu laptop hızlı işlemciye sahip, fakat bataryası çabuk bitiyor.",
  "references": {
    "detect": ["Bu laptop", "bataryası"],
    "sentimentKeywords": {
      "Bu laptop": ["hızlı işlemciye sahip"],
      "bataryası": ["çabuk bitiyor"]
    }
  }
},
{
  "prompts": "Onun tabletinin ekranı çok net, ama hoparlörleri zayıf.",
  "references": {
    "detect": ["Onun tabletinin ekranı", "hoparlörleri"],
    "sentimentKeywords": {
      "Onun tabletinin ekranı": ["çok net"],
      "hoparlörleri": ["zayıf"]
    }
  }
},
{
  "prompts": "Şu bilgisayarın tasarımı oldukça şık, ancak performansı ortalama.",
  "references": {
    "detect": ["Şu bilgisayarın tasarımı", "performansı"],
    "sentimentKeywords": {
      "Şu bilgisayarın tasarımı": ["oldukça şık"],
      "performansı": ["ortalama"]
    }
  }
},
{
  "prompts": "Bu hoparlörün sesi çok net, ama bas performansı zayıf.",
  "references": {
    "detect": ["Bu hoparlörün sesi", "bas performansı"],
    "sentimentKeywords": {
      "Bu hoparlörün sesi": ["çok net"],
      "bas performansı": ["zayıf"]
    }
  }
},
{
  "prompts": "O telefonun bataryası uzun ömürlü, ancak tasarımı eski moda.",
  "references": {
    "detect": ["O telefonun bataryası", "tasarımı"],
    "sentimentKeywords": {
      "O telefonun bataryası": ["uzun ömürlü"],
      "tasarımı": ["eski moda"]
    }
  }
},
{
  "prompts": "Bu ekran kartı yüksek performans sağlıyor, ama fiyatı çok yüksek.",
  "references": {
    "detect": ["Bu ekran kartı", "fiyatı"],
    "sentimentKeywords": {
      "Bu ekran kartı": ["yüksek performans"],
      "fiyatı": ["çok yüksek"]
    }
  }
},
{
  "prompts": "O kulaklığın konforu çok iyi, ancak ses kalitesi ortalama.",
  "references": {
    "detect": ["O kulaklığın konforu", "ses kalitesi"],
    "sentimentKeywords": {
      "O kulaklığın konforu": ["çok iyi"],
      "ses kalitesi": ["ortalama"]
    }
  }
},
{
  "prompts": "Şu yeni modelin bataryası çok uzun süre dayanıyor, ama fiyatı yüksek.",
  "references": {
    "detect": ["Şu yeni modelin bataryası", "fiyatı"],
    "sentimentKeywords": {
      "Şu yeni modelin bataryası": ["çok uzun süre dayanıyor"],
      "fiyatı": ["yüksek"]
    }
  }
},
{
  "prompts": "Bu akıllı saatin ekranı çok büyük, ancak şarj süresi kısa.",
  "references": {
    "detect": ["Bu akıllı saatin ekranı", "şarj süresi"],
    "sentimentKeywords": {
      "Bu akıllı saatin ekranı": ["çok büyük"],
      "şarj süresi": ["kısa"]
    }
  }
},
{
  "prompts": "O dizüstü bilgisayarın performansı mükemmel, fakat tasarımı çok kalın.",
  "references": {
    "detect": ["O dizüstü bilgisayarın performansı", "tasarımı"],
    "sentimentKeywords": {
      "O dizüstü bilgisayarın performansı": ["mükemmel"],
      "tasarımı": ["çok kalın"]
    }
  }
},
{
  "prompts": "Şu kamera mükemmel fotoğraflar çekiyor, ama video kalitesi yetersiz.",
  "references": {
    "detect": ["Şu kamera", "video kalitesi"],
    "sentimentKeywords": {
      "Şu kamera": ["mükemmel fotoğraflar çekiyor"],
      "video kalitesi": ["yetersiz"]
    }
  }
},{
    "prompts": "Turkcell'in internet hızı oldukça yüksek, Vodafone'un ise kapsama alanı geniş.",
    "references": {
      "detect": ["Turkcell'in internet hızı", "Vodafone'un kapsama alanı"],
      "sentimentKeywords": {
        "Turkcell'in internet hızı": ["oldukça yüksek"],
        "Vodafone'un kapsama alanı": ["geniş"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un müşteri hizmetleri çok yardımcı, Vodafone'un fiyatları ise biraz yüksek.",
    "references": {
      "detect": ["Türk Telekom'un müşteri hizmetleri", "Vodafone'un fiyatları"],
      "sentimentKeywords": {
        "Türk Telekom'un müşteri hizmetleri": ["çok yardımcı"],
        "Vodafone'un fiyatları": ["biraz yüksek"]
      }
    }
  },
  {
    "prompts": "Turkcell'in 5G hizmeti mükemmel, Türk Telekom'un ise bazen kesiliyor.",
    "references": {
      "detect": ["Turkcell'in 5G hizmeti", "Türk Telekom'un bazen kesilen hizmeti"],
      "sentimentKeywords": {
        "Turkcell'in 5G hizmeti": ["mükemmel"],
        "Türk Telekom'un bazen kesilen hizmeti": ["bazen kesiliyor"]
      }
    }
  },
  {
    "prompts": "Vodafone'un müşteri memnuniyeti çok yüksek, Turkcell'in faturalama sistemi ise karmaşık.",
    "references": {
      "detect": ["Vodafone'un müşteri memnuniyeti", "Turkcell'in faturalama sistemi"],
      "sentimentKeywords": {
        "Vodafone'un müşteri memnuniyeti": ["çok yüksek"],
        "Turkcell'in faturalama sistemi": ["karmaşık"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un internet paketleri uygun fiyatlı, Vodafone'un ise hız problemi var.",
    "references": {
      "detect": ["Türk Telekom'un internet paketleri", "Vodafone'un hız problemi"],
      "sentimentKeywords": {
        "Türk Telekom'un internet paketleri": ["uygun fiyatlı"],
        "Vodafone'un hız problemi": ["hız problemi var"]
      }
    }
  },
  {
    "prompts": "Turkcell'in mobil veri hizmetleri çok hızlı, Türk Telekom'un ise bazen yavaşlıyor.",
    "references": {
      "detect": ["Turkcell'in mobil veri hizmetleri", "Türk Telekom'un bazen yavaşlayan hizmeti"],
      "sentimentKeywords": {
        "Turkcell'in mobil veri hizmetleri": ["çok hızlı"],
        "Türk Telekom'un bazen yavaşlayan hizmeti": ["bazen yavaşlıyor"]
      }
    }
  },
  {
    "prompts": "Vodafone'un yurt dışı paketleri kullanışlı, Turkcell'in ise fiyatları yüksek.",
    "references": {
      "detect": ["Vodafone'un yurt dışı paketleri", "Turkcell'in fiyatları"],
      "sentimentKeywords": {
        "Vodafone'un yurt dışı paketleri": ["kullanışlı"],
        "Turkcell'in fiyatları": ["yüksek"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un telefon sinyali genellikle güçlü, Vodafone'un ise bazı bölgelerde zayıf.",
    "references": {
      "detect": ["Türk Telekom'un telefon sinyali", "Vodafone'un telefon sinyali"],
      "sentimentKeywords": {
        "Türk Telekom'un telefon sinyali": ["genellikle güçlü"],
        "Vodafone'un telefon sinyali": ["bazı bölgelerde zayıf"]
      }
    }
  },
  {
    "prompts": "Turkcell'in kampanya seçenekleri çok çeşitli, Türk Telekom'un ise müşteri desteği etkili.",
    "references": {
      "detect": ["Turkcell'in kampanya seçenekleri", "Türk Telekom'un müşteri desteği"],
      "sentimentKeywords": {
        "Turkcell'in kampanya seçenekleri": ["çok çeşitli"],
        "Türk Telekom'un müşteri desteği": ["etkili"]
      }
    }
  },
  {
    "prompts": "Vodafone'un 4.5G hizmeti yeterli, Turkcell'in ise veri kotası düşük.",
    "references": {
      "detect": ["Vodafone'un 4.5G hizmeti", "Turkcell'in veri kotası"],
      "sentimentKeywords": {
        "Vodafone'un 4.5G hizmeti": ["yeterli"],
        "Turkcell'in veri kotası": ["düşük"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un internet hızı oldukça iyi, Vodafone'un ise müşteri memnuniyeti düşük.",
    "references": {
      "detect": ["Türk Telekom'un internet hızı", "Vodafone'un müşteri memnuniyeti"],
      "sentimentKeywords": {
        "Türk Telekom'un internet hızı": ["oldukça iyi"],
        "Vodafone'un müşteri memnuniyeti": ["düşük"]
      }
    }
  },
  {
    "prompts": "Turkcell'in çağrı kalitesi mükemmel, Türk Telekom'un ise internet bağlantısı zaman zaman kesiliyor.",
    "references": {
      "detect": ["Turkcell'in çağrı kalitesi", "Türk Telekom'un internet bağlantısı"],
      "sentimentKeywords": {
        "Turkcell'in çağrı kalitesi": ["mükemmel"],
        "Türk Telekom'un internet bağlantısı": ["zaman zaman kesiliyor"]
      }
    }
  },
  {
    "prompts": "Vodafone'un telefon tarifeleri cazip, Turkcell'in ise internet servisleri yüksek kaliteli.",
    "references": {
      "detect": ["Vodafone'un telefon tarifeleri", "Turkcell'in internet servisleri"],
      "sentimentKeywords": {
        "Vodafone'un telefon tarifeleri": ["cazip"],
        "Turkcell'in internet servisleri": ["yüksek kaliteli"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un mobil uygulaması kullanışlı, Vodafone'un ise müşteri hizmetleri yetersiz.",
    "references": {
      "detect": ["Türk Telekom'un mobil uygulaması", "Vodafone'un müşteri hizmetleri"],
      "sentimentKeywords": {
        "Türk Telekom'un mobil uygulaması": ["kullanışlı"],
        "Vodafone'un müşteri hizmetleri": ["yetersiz"]
      }
    }
  },
  {
    "prompts": "Turkcell'in fiber interneti hızlı, Türk Telekom'un ise telefon görüşmeleri bazen kesiliyor.",
    "references": {
      "detect": ["Turkcell'in fiber interneti", "Türk Telekom'un telefon görüşmeleri"],
      "sentimentKeywords": {
        "Turkcell'in fiber interneti": ["hızlı"],
        "Türk Telekom'un telefon görüşmeleri": ["bazen kesiliyor"]
      }
    }
  },
  {
    "prompts": "Vodafone'un veri paketleri uygun fiyatlı, Turkcell'in ise internet hızı oldukça iyi.",
    "references": {
      "detect": ["Vodafone'un veri paketleri", "Turkcell'in internet hızı"],
      "sentimentKeywords": {
        "Vodafone'un veri paketleri": ["uygun fiyatlı"],
        "Turkcell'in internet hızı": ["oldukça iyi"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un tarifeleri rekabetçi, Vodafone'un ise müşteri destek hizmeti tatmin edici.",
    "references": {
      "detect": ["Türk Telekom'un tarifeleri", "Vodafone'un müşteri destek hizmeti"],
      "sentimentKeywords": {
        "Türk Telekom'un tarifeleri": ["rekabetçi"],
        "Vodafone'un müşteri destek hizmeti": ["tatmin edici"]
      }
    }
  },
  {
    "prompts": "Turkcell'in mobil internet hızı yüksek, Türk Telekom'un ise kampanya seçenekleri sınırlı.",
    "references": {
      "detect": ["Turkcell'in mobil internet hızı", "Türk Telekom'un kampanya seçenekleri"],
      "sentimentKeywords": {
        "Turkcell'in mobil internet hızı": ["yüksek"],
        "Türk Telekom'un kampanya seçenekleri": ["sınırlı"]
      }
    }
  },
  {
    "prompts": "Vodafone'un 5G teknolojisi kullanışlı, Turkcell'in ise faturalama sistemi bazen karmaşık.",
    "references": {
      "detect": ["Vodafone'un 5G teknolojisi", "Turkcell'in faturalama sistemi"],
      "sentimentKeywords": {
        "Vodafone'un 5G teknolojisi": ["kullanışlı"],
        "Turkcell'in faturalama sistemi": ["bazen karmaşık"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un sinyal kalitesi genellikle yüksek, Vodafone'un ise internet servisleri bazen yavaş.",
    "references": {
      "detect": ["Türk Telekom'un sinyal kalitesi", "Vodafone'un internet servisleri"],
      "sentimentKeywords": {
        "Türk Telekom'un sinyal kalitesi": ["genellikle yüksek"],
        "Vodafone'un internet servisleri": ["bazen yavaş"]
      }
    }
  },
  {
    "prompts": "Turkcell'in müşteri hizmetleri hızlı cevap veriyor, Türk Telekom'un ise veri paketleri yetersiz.",
    "references": {
      "detect": ["Turkcell'in müşteri hizmetleri", "Türk Telekom'un veri paketleri"],
      "sentimentKeywords": {
        "Turkcell'in müşteri hizmetleri": ["hızlı cevap veriyor"],
        "Türk Telekom'un veri paketleri": ["yetersiz"]
      }
    }
  },
  {
    "prompts": "Vodafone'un telefon sinyali güçlü, Turkcell'in ise internet hızı ortalama.",
    "references": {
      "detect": ["Vodafone'un telefon sinyali", "Turkcell'in internet hızı"],
      "sentimentKeywords": {
        "Vodafone'un telefon sinyali": ["güçlü"],
        "Turkcell'in internet hızı": ["ortalama"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un mobil interneti hızlı, Vodafone'un ise müşteri hizmetleri etkili değil.",
    "references": {
      "detect": ["Türk Telekom'un mobil interneti", "Vodafone'un müşteri hizmetleri"],
      "sentimentKeywords": {
        "Türk Telekom'un mobil interneti": ["hızlı"],
        "Vodafone'un müşteri hizmetleri": ["etkili değil"]
      }
    }
  },
  {
    "prompts": "Turkcell'in fiyatları uygun, Türk Telekom'un ise sinyal kalitesi zaman zaman düşüyor.",
    "references": {
      "detect": ["Turkcell'in fiyatları", "Türk Telekom'un sinyal kalitesi"],
      "sentimentKeywords": {
        "Turkcell'in fiyatları": ["uygun"],
        "Türk Telekom'un sinyal kalitesi": ["zaman zaman düşüyor"]
      }
    }
  },
  {
    "prompts": "Vodafone'un mobil veri hizmetleri kullanışlı, Turkcell'in ise müşteri desteği çok iyi.",
    "references": {
      "detect": ["Vodafone'un mobil veri hizmetleri", "Turkcell'in müşteri desteği"],
      "sentimentKeywords": {
        "Vodafone'un mobil veri hizmetleri": ["kullanışlı"],
        "Turkcell'in müşteri desteği": ["çok iyi"]
      }
    }
  },
  {
    "prompts": "Türk Telekom'un internet hızları genellikle yüksek, Vodafone'un ise fiyatları çok cazip.",
    "references": {
      "detect": ["Türk Telekom'un internet hızları", "Vodafone'un fiyatları"],
      "sentimentKeywords": {
        "Türk Telekom'un internet hızları": ["genellikle yüksek"],
        "Vodafone'un fiyatları": ["çok cazip"]
      }
    }
  },

  {
    "prompts": "Turkcell’in 5G bağlantısı gerçekten harika, ancak bazen sinyal kaybı yaşanabiliyor.",
    "references": {
      "detect": ["Turkcell’in 5G bağlantısı", "sinyal kaybı"],
      "sentimentKeywords": {
        "Turkcell’in 5G bağlantısı": ["harika"],
        "sinyal kaybı": ["bazı zamanlar yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un müşteri hizmetlerine ulaşmak çok kolay, ama internet hızında zaman zaman düşüşler oluyor.",
    "references": {
      "detect": ["Türk Telekom’un müşteri hizmetleri", "internet hızı"],
      "sentimentKeywords": {
        "Türk Telekom’un müşteri hizmetleri": ["çok kolay ulaşılabilir"],
        "internet hızı": ["zaman zaman düşüşler oluyor"]
      }
    }
  },
  {
    "prompts": "Vodafone’un yurt içi tarifeleri oldukça uygun, fakat uluslararası görüşmeler biraz pahalı.",
    "references": {
      "detect": ["Vodafone’un yurt içi tarifeleri", "uluslararası görüşmeler"],
      "sentimentKeywords": {
        "Vodafone’un yurt içi tarifeleri": ["uygun"],
        "uluslararası görüşmeler": ["biraz pahalı"]
      }
    }
  },
  {
    "prompts": "Turkcell’in mobil uygulaması çok kullanışlı, ama bazen giriş sorunları yaşıyorum.",
    "references": {
      "detect": ["Turkcell’in mobil uygulaması", "giriş sorunları"],
      "sentimentKeywords": {
        "Turkcell’in mobil uygulaması": ["çok kullanışlı"],
        "giriş sorunları": ["bazen yaşıyorum"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un tarifeleri oldukça rekabetçi, ancak müşteri hizmetleriyle ilgili sıkıntılar yaşıyorum.",
    "references": {
      "detect": ["Türk Telekom’un tarifeleri", "müşteri hizmetleri"],
      "sentimentKeywords": {
        "Türk Telekom’un tarifeleri": ["rekabetçi"],
        "müşteri hizmetleri": ["sıkıntılar yaşıyorum"]
      }
    }
  },
  {
    "prompts": "Vodafone’un sinyal kalitesi iyi, ama internet hızı bazen beklediğimden düşük.",
    "references": {
      "detect": ["Vodafone’un sinyal kalitesi", "internet hızı"],
      "sentimentKeywords": {
        "Vodafone’un sinyal kalitesi": ["iyi"],
        "internet hızı": ["bazen düşük"]
      }
    }
  },
  {
    "prompts": "Turkcell’in veri paketleri gerçekten yeterli, fakat müşteri desteği daha iyi olabilir.",
    "references": {
      "detect": ["Turkcell’in veri paketleri", "müşteri desteği"],
      "sentimentKeywords": {
        "Turkcell’in veri paketleri": ["gerçekten yeterli"],
        "müşteri desteği": ["daha iyi olabilir"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un internet hızı çoğunlukla tatmin edici, ancak bazı bölgelerde sorun yaşayabiliyorum.",
    "references": {
      "detect": ["Türk Telekom’un internet hızı", "bazı bölgelerde sorun"],
      "sentimentKeywords": {
        "Türk Telekom’un internet hızı": ["tatmin edici"],
        "bazı bölgelerde sorun": ["yaşayabiliyorum"]
      }
    }
  },
  {
    "prompts": "Vodafone’un 4.5G hizmeti gerçekten iyi, ama müşteri destek ekibi daha hızlı olabilir.",
    "references": {
      "detect": ["Vodafone’un 4.5G hizmeti", "müşteri destek ekibi"],
      "sentimentKeywords": {
        "Vodafone’un 4.5G hizmeti": ["gerçekten iyi"],
        "müşteri destek ekibi": ["daha hızlı olabilir"]
      }
    }
  },
  {
    "prompts": "Turkcell’in fiyatları makul, fakat internet hızında bazı zamanlar dalgalanmalar olabiliyor.",
    "references": {
      "detect": ["Turkcell’in fiyatları", "internet hızı"],
      "sentimentKeywords": {
        "Turkcell’in fiyatları": ["makul"],
        "internet hızı": ["bazı zamanlar dalgalanma"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un mobil interneti genellikle iyi, ancak bazı günlerde bağlantı sorunları yaşıyorum.",
    "references": {
      "detect": ["Türk Telekom’un mobil interneti", "bağlantı sorunları"],
      "sentimentKeywords": {
        "Türk Telekom’un mobil interneti": ["genellikle iyi"],
        "bağlantı sorunları": ["bazı günlerde yaşıyorum"]
      }
    }
  },
  {
    "prompts": "Vodafone’un telefon sinyali genelde güçlü, fakat bazen kapalı alanlarda çekmiyor.",
    "references": {
      "detect": ["Vodafone’un telefon sinyali", "kapalı alanlarda çekme"],
      "sentimentKeywords": {
        "Vodafone’un telefon sinyali": ["genelde güçlü"],
        "kapalı alanlarda çekme": ["bazen çekmiyor"]
      }
    }
  },
  {
    "prompts": "Turkcell’in müşteri hizmetleri oldukça hızlı, ama bazen faturalama hataları olabiliyor.",
    "references": {
      "detect": ["Turkcell’in müşteri hizmetleri", "faturalama hataları"],
      "sentimentKeywords": {
        "Turkcell’in müşteri hizmetleri": ["oldukça hızlı"],
        "faturalama hataları": ["bazen olabiliyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un kampanya seçenekleri geniş, fakat internet hızında sorunlar yaşanabiliyor.",
    "references": {
      "detect": ["Türk Telekom’un kampanya seçenekleri", "internet hızı"],
      "sentimentKeywords": {
        "Türk Telekom’un kampanya seçenekleri": ["geniş"],
        "internet hızı": ["sorunlar yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "Vodafone’un veri kotası yeterli, ama internet hızı bazı zamanlarda yavaşlıyor.",
    "references": {
      "detect": ["Vodafone’un veri kotası", "internet hızı"],
      "sentimentKeywords": {
        "Vodafone’un veri kotası": ["yeterli"],
        "internet hızı": ["bazen yavaşlıyor"]
      }
    }
  },
  {
    "prompts": "Turkcell’in 4.5G bağlantısı çok hızlı, fakat müşteri hizmetleri yanıt süresi biraz uzun.",
    "references": {
      "detect": ["Turkcell’in 4.5G bağlantısı", "müşteri hizmetleri yanıt süresi"],
      "sentimentKeywords": {
        "Turkcell’in 4.5G bağlantısı": ["çok hızlı"],
        "müşteri hizmetleri yanıt süresi": ["biraz uzun"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un sinyal kalitesi iyi, ama internet bağlantısı bazen kopabiliyor.",
    "references": {
      "detect": ["Türk Telekom’un sinyal kalitesi", "internet bağlantısı"],
      "sentimentKeywords": {
        "Türk Telekom’un sinyal kalitesi": ["iyi"],
        "internet bağlantısı": ["bazen kopabiliyor"]
      }
    }
  },
  {
    "prompts": "Vodafone’un fiyatları oldukça cazip, fakat internet servisleri bazen yavaş olabiliyor.",
    "references": {
      "detect": ["Vodafone’un fiyatları", "internet servisleri"],
      "sentimentKeywords": {
        "Vodafone’un fiyatları": ["cazip"],
        "internet servisleri": ["bazen yavaş"]
      }
    }
  },
  {
    "prompts": "Turkcell’in müşteri desteği çok yardımcı, fakat internet paketleri biraz daha uygun olabilir.",
    "references": {
      "detect": ["Turkcell’in müşteri desteği", "internet paketleri"],
      "sentimentKeywords": {
        "Turkcell’in müşteri desteği": ["çok yardımcı"],
        "internet paketleri": ["biraz daha uygun olabilir"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un internet hızından memnunum, ama sinyal kalitesi bazı yerlerde düşük.",
    "references": {
      "detect": ["Türk Telekom’un internet hızı", "sinyal kalitesi"],
      "sentimentKeywords": {
        "Türk Telekom’un internet hızı": ["memnunum"],
        "sinyal kalitesi": ["bazı yerlerde düşük"]
      }
    }
  },
  {
    "prompts": "Vodafone’un müşteri hizmetleri genellikle hızlı, fakat fiyatlandırmada daha fazla seçenek olmalı.",
    "references": {
      "detect": ["Vodafone’un müşteri hizmetleri", "fiyatlandırma seçenekleri"],
      "sentimentKeywords": {
        "Vodafone’un müşteri hizmetleri": ["genellikle hızlı"],
        "fiyatlandırma seçenekleri": ["daha fazla olmalı"]
      }
    }
  },
  {
    "prompts": "Turkcell’in internet servisleri genellikle iyi, ama bazı zamanlar hız düşüşü yaşıyorum.",
    "references": {
      "detect": ["Turkcell’in internet servisleri", "hız düşüşü"],
      "sentimentKeywords": {
        "Turkcell’in internet servisleri": ["genellikle iyi"],
        "hız düşüşü": ["bazı zamanlar yaşıyorum"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un sinyal gücü tatmin edici, ancak internet bağlantısı bazen kesiliyor.",
    "references": {
      "detect": ["Türk Telekom’un sinyal gücü", "internet bağlantısı"],
      "sentimentKeywords": {
        "Türk Telekom’un sinyal gücü": ["tatmin edici"],
        "internet bağlantısı": ["bazen kesiliyor"]
      }
    }
  },
  {
    "prompts": "Vodafone’un veri paketleri geniş, ama internet hızı bazı bölgelerde düşük olabiliyor.",
    "references": {
      "detect": ["Vodafone’un veri paketleri", "internet hızı"],
      "sentimentKeywords": {
        "Vodafone’un veri paketleri": ["geniş"],
        "internet hızı": ["bazı bölgelerde düşük"]
      }
    }
  },
  {
    "prompts": "Turkcell’in kampanya fırsatları oldukça cazip, fakat internet hızında sıkıntılar yaşanabiliyor.",
    "references": {
      "detect": ["Turkcell’in kampanya fırsatları", "internet hızı"],
      "sentimentKeywords": {
        "Turkcell’in kampanya fırsatları": ["cazip"],
        "internet hızı": ["sıkıntılar yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un müşteri desteği iyi, fakat fatura düzenlemesi konusunda problemler yaşanabiliyor.",
    "references": {
      "detect": ["Türk Telekom’un müşteri desteği", "fatura düzenlemesi"],
      "sentimentKeywords": {
        "Türk Telekom’un müşteri desteği": ["iyi"],
        "fatura düzenlemesi": ["problemler yaşanabiliyor"]
      }
    }
  },{
    "prompts": "Türknet’in internet bağlantısı oldukça stabil, ancak bazı bölgelerde bağlantı hızında düşüş yaşanabiliyor.",
    "references": {
      "detect": ["Türknet’in internet bağlantısı", "bağlantı hızı"],
      "sentimentKeywords": {
        "Türknet’in internet bağlantısı": ["oldukça stabil"],
        "bağlantı hızı": ["bazı bölgelerde düşüş yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un TV hizmetleri geniş kanal seçenekleri sunuyor, fakat bazı kanallarda sinyal sorunları yaşanabiliyor.",
    "references": {
      "detect": ["Türk Telekom’un TV hizmetleri", "sinyal sorunları"],
      "sentimentKeywords": {
        "Türk Telekom’un TV hizmetleri": ["geniş kanal seçenekleri sunuyor"],
        "sinyal sorunları": ["bazı kanallarda yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "D-Smart’ın internet planları oldukça cazip, ancak bazı müşteriler hızda dalgalanma yaşadıklarını belirtiyor.",
    "references": {
      "detect": ["D-Smart’ın internet planları", "hız dalgalanması"],
      "sentimentKeywords": {
        "D-Smart’ın internet planları": ["cazip"],
        "hız dalgalanması": ["bazı müşteriler yaşadıklarını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Superonline’ın müşteri hizmetleri çok ilgili, fakat bazen teknik sorunların çözümü zaman alabiliyor.",
    "references": {
      "detect": ["Superonline’ın müşteri hizmetleri", "teknik sorunlar"],
      "sentimentKeywords": {
        "Superonline’ın müşteri hizmetleri": ["çok ilgili"],
        "teknik sorunlar": ["zaman alabiliyor"]
      }
    }
  },
  {
    "prompts": "Türknet’in fiber internet hızı oldukça yüksek, ancak bazı kullanıcılar bağlantı kopmaları yaşadıklarını söylüyor.",
    "references": {
      "detect": ["Türknet’in fiber internet hızı", "bağlantı kopmaları"],
      "sentimentKeywords": {
        "Türknet’in fiber internet hızı": ["oldukça yüksek"],
        "bağlantı kopmaları": ["bazı kullanıcılar yaşıyor"]
      }
    }
  },
  {
    "prompts": "Bip’in sesli arama hizmetleri etkili, ama bazı kullanıcılar uygulamanın donma sorunları yaşadığını belirtiyor.",
    "references": {
      "detect": ["Bip’in sesli arama hizmetleri", "uygulama donma sorunları"],
      "sentimentKeywords": {
        "Bip’in sesli arama hizmetleri": ["etkili"],
        "uygulama donma sorunları": ["bazı kullanıcılar yaşadığını belirtiyor"]
      }
    }
  },
  {
    "prompts": "D-Smart’ın fiber interneti genel olarak hızlı, ancak bazı bölgelerde hızın düşebildiği belirtiliyor.",
    "references": {
      "detect": ["D-Smart’ın fiber interneti", "hız düşüşü"],
      "sentimentKeywords": {
        "D-Smart’ın fiber interneti": ["genel olarak hızlı"],
        "hız düşüşü": ["bazı bölgelerde yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un müşteri destek ekibi oldukça yardımcı, ancak bazı kullanıcılar işlem sürelerinin uzun olduğundan şikayetçi.",
    "references": {
      "detect": ["Türk Telekom’un müşteri destek ekibi", "işlem süreleri"],
      "sentimentKeywords": {
        "Türk Telekom’un müşteri destek ekibi": ["oldukça yardımcı"],
        "işlem süreleri": ["bazı kullanıcılar uzun olduğundan şikayetçi"]
      }
    }
  },
  {
    "prompts": "Türknet’in mobil internet paketleri geniş veri seçenekleri sunuyor, fakat bazı bölgelerde sinyal zayıflığı yaşanabiliyor.",
    "references": {
      "detect": ["Türknet’in mobil internet paketleri", "sinyal zayıflığı"],
      "sentimentKeywords": {
        "Türknet’in mobil internet paketleri": ["geniş veri seçenekleri sunuyor"],
        "sinyal zayıflığı": ["bazı bölgelerde yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "D-Smart’ın TV ve internet paketleri bir arada iyi bir değer sunuyor, fakat bazı kullanıcılar fatura düzenlemelerinde sorun yaşadıklarını söylüyor.",
    "references": {
      "detect": ["D-Smart’ın TV ve internet paketleri", "fatura düzenlemeleri"],
      "sentimentKeywords": {
        "D-Smart’ın TV ve internet paketleri": ["iyi bir değer sunuyor"],
        "fatura düzenlemeleri": ["bazı kullanıcılar sorun yaşadıklarını söylüyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un 4.5G hizmetleri hızlı ve güvenilir, ancak bazı kırsal alanlarda sinyal sıkıntısı yaşanabiliyor.",
    "references": {
      "detect": ["Türk Telekom’un 4.5G hizmetleri", "sinyal sıkıntısı"],
      "sentimentKeywords": {
        "Türk Telekom’un 4.5G hizmetleri": ["hızlı ve güvenilir"],
        "sinyal sıkıntısı": ["bazı kırsal alanlarda yaşanabiliyor"]
      }
    }
  },
  {
    "prompts": "Superonline’ın müşteri desteği hızlı ve çözüm odaklı, fakat bazı teknik sorunlarda çözüm süresi uzun olabiliyor.",
    "references": {
      "detect": ["Superonline’ın müşteri desteği", "teknik sorunlar"],
      "sentimentKeywords": {
        "Superonline’ın müşteri desteği": ["hızlı ve çözüm odaklı"],
        "teknik sorunlar": ["bazı sorunlarda çözüm süresi uzun olabiliyor"]
      }
    }
  },
  {
    "prompts": "Türknet’in TV platformu çeşitli kanal seçenekleri sunuyor, ancak bazı kullanıcılar sinyal kalitesinden memnun değil.",
    "references": {
      "detect": ["Türknet’in TV platformu", "sinyal kalitesi"],
      "sentimentKeywords": {
        "Türknet’in TV platformu": ["çeşitli kanal seçenekleri sunuyor"],
        "sinyal kalitesi": ["bazı kullanıcılar memnun değil"]
      }
    }
  },
  {
    "prompts": "Bip’in grup sohbetleri çok kullanışlı, ancak bazı kullanıcılar uygulamanın zaman zaman yavaşladığını belirtiyor.",
    "references": {
      "detect": ["Bip’in grup sohbetleri", "uygulamanın yavaşlaması"],
      "sentimentKeywords": {
        "Bip’in grup sohbetleri": ["çok kullanışlı"],
        "uygulamanın yavaşlaması": ["bazı kullanıcılar belirtiyor"]
      }
    }
  },
  {
    "prompts": "D-Smart’ın müşteri hizmetleri genellikle hızlı, fakat bazı kullanıcılar fatura hatalarından şikayetçi.",
    "references": {
      "detect": ["D-Smart’ın müşteri hizmetleri", "fatura hataları"],
      "sentimentKeywords": {
        "D-Smart’ın müşteri hizmetleri": ["genellikle hızlı"],
        "fatura hataları": ["bazı kullanıcılar şikayetçi"]
      }
    }
  },
  {
    "prompts": "Türknet’in fiber interneti çok hızlı, ancak bazı yerlerde müşteri destek hizmetlerinin yetersiz olduğunu düşünenler var.",
    "references": {
      "detect": ["Türknet’in fiber interneti", "müşteri destek hizmetleri"],
      "sentimentKeywords": {
        "Türknet’in fiber interneti": ["çok hızlı"],
        "müşteri destek hizmetleri": ["bazı yerlerde yetersiz"]
      }
    }
  },
  {
    "prompts": "Superonline’ın internet bağlantısı genellikle stabil, fakat bazı kullanıcılar yavaşlama sorunları yaşadıklarını belirtiyor.",
    "references": {
      "detect": ["Superonline’ın internet bağlantısı", "yavaşlama sorunları"],
      "sentimentKeywords": {
        "Superonline’ın internet bağlantısı": ["genellikle stabil"],
        "yavaşlama sorunları": ["bazı kullanıcılar yaşadıklarını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un internet hızları tatmin edici, ancak bazı kullanıcılar fatura düzenlemeleri konusunda zorluk yaşadıklarını belirtiyor.",
    "references": {
      "detect": ["Türk Telekom’un internet hızları", "fatura düzenlemeleri"],
      "sentimentKeywords": {
        "Türk Telekom’un internet hızları": ["tatmin edici"],
        "fatura düzenlemeleri": ["bazı kullanıcılar zorluk yaşadıklarını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Bip’in video görüşme kalitesi genel olarak iyi, fakat bazı kullanıcılar ses kesilme sorunları yaşadığını belirtiyor.",
    "references": {
      "detect": ["Bip’in video görüşme kalitesi", "ses kesilme sorunları"],
      "sentimentKeywords": {
        "Bip’in video görüşme kalitesi": ["genel olarak iyi"],
        "ses kesilme sorunları": ["bazı kullanıcılar yaşadığını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Türknet’in kampanya fırsatları cazip, ancak bazı bölgelerde hizmet kalitesi beklentilerin altında kalabiliyor.",
    "references": {
      "detect": ["Türknet’in kampanya fırsatları", "hizmet kalitesi"],
      "sentimentKeywords": {
        "Türknet’in kampanya fırsatları": ["cazip"],
        "hizmet kalitesi": ["bazı bölgelerde beklentilerin altında kalabiliyor"]
      }
    }
  },
  {
    "prompts": "D-Smart’ın internet paketleri iyi, ancak bazı kullanıcılar hız testlerinde düşük sonuçlar aldıklarını belirtiyor.",
    "references": {
      "detect": ["D-Smart’ın internet paketleri", "hız testleri"],
      "sentimentKeywords": {
        "D-Smart’ın internet paketleri": ["iyi"],
        "hız testleri": ["bazı kullanıcılar düşük sonuçlar aldıklarını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Türk Telekom’un mobil hizmetleri genelde iyi, fakat bazı kullanıcılar yüksek fatura ücretlerinden şikayetçi.",
    "references": {
      "detect": ["Türk Telekom’un mobil hizmetleri", "yüksek fatura ücretleri"],
      "sentimentKeywords": {
        "Türk Telekom’un mobil hizmetleri": ["genelde iyi"],
        "yüksek fatura ücretleri": ["bazı kullanıcılar şikayetçi"]
      }
    }
  },
  {
    "prompts": "Superonline’ın müşteri destek ekibi genelde hızlı, fakat bazı sorunlarda daha etkili bir çözüm bulunabiliyor.",
    "references": {
      "detect": ["Superonline’ın müşteri destek ekibi", "sorun çözümü"],
      "sentimentKeywords": {
        "Superonline’ın müşteri destek ekibi": ["genelde hızlı"],
        "sorun çözümü": ["bazı sorunlarda daha etkili bir çözüm bulunabiliyor"]
      }
    }
  },
  {
    "prompts": "Türknet’in TV ve internet paketleri uygun fiyatlı, fakat bazı kullanıcılar sinyal kalitesinin düşük olduğunu belirtiyor.",
    "references": {
      "detect": ["Türknet’in TV ve internet paketleri", "sinyal kalitesi"],
      "sentimentKeywords": {
        "Türknet’in TV ve internet paketleri": ["uygun fiyatlı"],
        "sinyal kalitesi": ["bazı kullanıcılar düşük olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "D-Smart’ın müşteri destek ekibi genelde etkili, ancak bazı kullanıcılar çözüm süreçlerinin uzun sürdüğünü ifade ediyor.",
    "references": {
      "detect": ["D-Smart’ın müşteri destek ekibi", "çözüm süreçleri"],
      "sentimentKeywords": {
        "D-Smart’ın müşteri destek ekibi": ["genelde etkili"],
        "çözüm süreçleri": ["bazı kullanıcılar uzun sürdüğünü ifade ediyor"]
      }
    }
  },
 {
    "prompts": "Ooredoo’nun mobil internet hizmetleri genellikle hızlı, ancak bazı kullanıcılar kapsama alanının sınırlı olduğunu belirtiyor.",
    "references": {
      "detect": ["Ooredoo’nun mobil internet hizmetleri", "kapsama alanı"],
      "sentimentKeywords": {
        "Ooredoo’nun mobil internet hizmetleri": ["genellikle hızlı"],
        "kapsama alanı": ["bazı kullanıcılar sınırlı olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "Telenor’un telefon hizmetleri iyi performans gösteriyor, fakat bazı kullanıcılar müşteri hizmetlerinin yetersiz olduğunu ifade ediyor.",
    "references": {
      "detect": ["Telenor’un telefon hizmetleri", "müşteri hizmetleri"],
      "sentimentKeywords": {
        "Telenor’un telefon hizmetleri": ["iyi performans gösteriyor"],
        "müşteri hizmetleri": ["bazı kullanıcılar yetersiz olduğunu ifade ediyor"]
      }
    }
  },
  {
    "prompts": "Vodafone'un uluslararası roaming hizmetleri genellikle faydalı, fakat bazı kullanıcılar ücretlerin yüksek olduğunu belirtiyor.",
    "references": {
      "detect": ["Vodafone'un uluslararası roaming hizmetleri", "ücretler"],
      "sentimentKeywords": {
        "Vodafone'un uluslararası roaming hizmetleri": ["genellikle faydalı"],
        "ücretler": ["bazı kullanıcılar yüksek olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "AT&T'nin mobil veri hizmetleri genellikle tatmin edici, ancak bazı kullanıcılar veri hızının bazı bölgelerde düşük olduğunu söylüyor.",
    "references": {
      "detect": ["AT&T'nin mobil veri hizmetleri", "veri hızı"],
      "sentimentKeywords": {
        "AT&T'nin mobil veri hizmetleri": ["genellikle tatmin edici"],
        "veri hızı": ["bazı kullanıcılar bazı bölgelerde düşük olduğunu söylüyor"]
      }
    }
  },
  {
    "prompts": "Deutsche Telekom'un fiber internet hizmetleri genellikle hızlı, ancak bazı kullanıcılar müşteri desteklerinin etkisiz olduğunu belirtiyor.",
    "references": {
      "detect": ["Deutsche Telekom'un fiber internet hizmetleri", "müşteri destek"],
      "sentimentKeywords": {
        "Deutsche Telekom'un fiber internet hizmetleri": ["genellikle hızlı"],
        "müşteri destek": ["bazı kullanıcılar etkisiz olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "Orange’ın internet hizmetleri genellikle iyi, ancak bazı kullanıcılar bağlantı kesintileri yaşadıklarını ifade ediyor.",
    "references": {
      "detect": ["Orange’ın internet hizmetleri", "bağlantı kesintileri"],
      "sentimentKeywords": {
        "Orange’ın internet hizmetleri": ["genellikle iyi"],
        "bağlantı kesintileri": ["bazı kullanıcılar yaşadıklarını ifade ediyor"]
      }
    }
  },
  {
    "prompts": "Telia’nın mobil uygulama performansı genelde tatmin edici, fakat bazı kullanıcılar uygulamanın yavaş çalıştığını belirtiyor.",
    "references": {
      "detect": ["Telia’nın mobil uygulama performansı", "uygulama hızı"],
      "sentimentKeywords": {
        "Telia’nın mobil uygulama performansı": ["genelde tatmin edici"],
        "uygulama hızı": ["bazı kullanıcılar yavaş çalıştığını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Verizon’un ev internet hizmetleri genellikle yüksek hızlı, ancak bazı kullanıcılar modem kalitesinden memnun değil.",
    "references": {
      "detect": ["Verizon’un ev internet hizmetleri", "modem kalitesi"],
      "sentimentKeywords": {
        "Verizon’un ev internet hizmetleri": ["genellikle yüksek hızlı"],
        "modem kalitesi": ["bazı kullanıcılar memnun değil"]
      }
    }
  },
  {
    "prompts": "Sprint’in mobil veri hızları genellikle yeterli, fakat bazı kullanıcılar bağlantı kalitesinin zaman zaman düşük olduğunu belirtiyor.",
    "references": {
      "detect": ["Sprint’in mobil veri hızları", "bağlantı kalitesi"],
      "sentimentKeywords": {
        "Sprint’in mobil veri hızları": ["genellikle yeterli"],
        "bağlantı kalitesi": ["bazı kullanıcılar zaman zaman düşük olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "China Mobile’ın uluslararası hizmetleri genellikle faydalı, ancak bazı kullanıcılar hizmet ücretlerinin yüksek olduğunu belirtiyor.",
    "references": {
      "detect": ["China Mobile’ın uluslararası hizmetleri", "hizmet ücretleri"],
      "sentimentKeywords": {
        "China Mobile’ın uluslararası hizmetleri": ["genellikle faydalı"],
        "hizmet ücretleri": ["bazı kullanıcılar yüksek olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "Telecom Italia’nın internet hızları genellikle iyi, ancak bazı kullanıcılar bağlantı kesintilerinden şikayetçi.",
    "references": {
      "detect": ["Telecom Italia’nın internet hızları", "bağlantı kesintileri"],
      "sentimentKeywords": {
        "Telecom Italia’nın internet hızları": ["genellikle iyi"],
        "bağlantı kesintileri": ["bazı kullanıcılar şikayetçi"]
      }
    }
  },
  {
    "prompts": "Rogers’in müşteri destek hizmetleri genellikle tatmin edici, fakat bazı kullanıcılar uzun bekleme sürelerinden şikayetçi.",
    "references": {
      "detect": ["Rogers’in müşteri destek hizmetleri", "bekleme süreleri"],
      "sentimentKeywords": {
        "Rogers’in müşteri destek hizmetleri": ["genellikle tatmin edici"],
        "bekleme süreleri": ["bazı kullanıcılar uzun olduğundan şikayetçi"]
      }
    }
  },
  {
    "prompts": "Singtel’in mobil hizmetleri genellikle iyi, ancak bazı kullanıcılar kapsama alanının yetersiz olduğunu belirtiyor.",
    "references": {
      "detect": ["Singtel’in mobil hizmetleri", "kapsama alanı"],
      "sentimentKeywords": {
        "Singtel’in mobil hizmetleri": ["genellikle iyi"],
        "kapsama alanı": ["bazı kullanıcılar yetersiz olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "Bell’in internet hizmetleri genellikle hızlı, fakat bazı kullanıcılar müşteri desteğinin yetersiz olduğunu ifade ediyor.",
    "references": {
      "detect": ["Bell’in internet hizmetleri", "müşteri desteği"],
      "sentimentKeywords": {
        "Bell’in internet hizmetleri": ["genellikle hızlı"],
        "müşteri desteği": ["bazı kullanıcılar yetersiz olduğunu ifade ediyor"]
      }
    }
  },
  {
    "prompts": "Telstra’nın mobil veri hizmetleri genelde hızlı, fakat bazı kullanıcılar veri hızının bazı bölgelerde düşük olduğunu belirtiyor.",
    "references": {
      "detect": ["Telstra’nın mobil veri hizmetleri", "veri hızı"],
      "sentimentKeywords": {
        "Telstra’nın mobil veri hizmetleri": ["genelde hızlı"],
        "veri hızı": ["bazı kullanıcılar bazı bölgelerde düşük olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "Vodafone’ın fiber internet hizmetleri genellikle hızlı, ancak bazı kullanıcılar hizmet kesintilerinden şikayetçi.",
    "references": {
      "detect": ["Vodafone’ın fiber internet hizmetleri", "hizmet kesintileri"],
      "sentimentKeywords": {
        "Vodafone’ın fiber internet hizmetleri": ["genellikle hızlı"],
        "hizmet kesintileri": ["bazı kullanıcılar şikayetçi"]
      }
    }
  },
  {
    "prompts": "Orange’ın mobil uygulama performansı genelde tatmin edici, fakat bazı kullanıcılar uygulamanın zaman zaman yavaş çalıştığını belirtiyor.",
    "references": {
      "detect": ["Orange’ın mobil uygulama performansı", "uygulama hızı"],
      "sentimentKeywords": {
        "Orange’ın mobil uygulama performansı": ["genelde tatmin edici"],
        "uygulama hızı": ["bazı kullanıcılar zaman zaman yavaş çalıştığını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Telefonica’nın uluslararası hizmetleri faydalı, ancak bazı kullanıcılar ücretlerin yüksek olduğunu belirtiyor.",
    "references": {
      "detect": ["Telefonica’nın uluslararası hizmetleri", "hizmet ücretleri"],
      "sentimentKeywords": {
        "Telefonica’nın uluslararası hizmetleri": ["faydalı"],
        "hizmet ücretleri": ["bazı kullanıcılar yüksek olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "T-Mobile’ın ev internet hizmetleri genellikle hızlı, fakat bazı kullanıcılar modem kalitesinden memnun değil.",
    "references": {
      "detect": ["T-Mobile’ın ev internet hizmetleri", "modem kalitesi"],
      "sentimentKeywords": {
        "T-Mobile’ın ev internet hizmetleri": ["genellikle hızlı"],
        "modem kalitesi": ["bazı kullanıcılar memnun değil"]
      }
    }
  },
  {
    "prompts": "Sprint’in uluslararası roaming hizmetleri genellikle faydalı, ancak bazı kullanıcılar yüksek ücretler ödediğinden yakınıyor.",
    "references": {
      "detect": ["Sprint’in uluslararası roaming hizmetleri", "ücretler"],
      "sentimentKeywords": {
        "Sprint’in uluslararası roaming hizmetleri": ["genellikle faydalı"],
        "ücretler": ["bazı kullanıcılar yüksek ödediğinden yakınıyor"]
      }
    }
  },
  {
    "prompts": "KPN’nin mobil veri hızları genellikle iyi, fakat bazı kullanıcılar kapsama alanının yetersiz olduğunu belirtiyor.",
    "references": {
      "detect": ["KPN’nin mobil veri hızları", "kapsama alanı"],
      "sentimentKeywords": {
        "KPN’nin mobil veri hızları": ["genellikle iyi"],
        "kapsama alanı": ["bazı kullanıcılar yetersiz olduğunu belirtiyor"]
      }
    }
  },
  {
    "prompts": "Bouygues’in internet hizmetleri genelde iyi performans gösteriyor, ancak bazı kullanıcılar müşteri desteği konusunda sorun yaşadıklarını belirtiyor.",
    "references": {
      "detect": ["Bouygues’in internet hizmetleri", "müşteri desteği"],
      "sentimentKeywords": {
        "Bouygues’in internet hizmetleri": ["genelde iyi performans gösteriyor"],
        "müşteri desteği": ["bazı kullanıcılar sorun yaşadıklarını belirtiyor"]
      }
    }
  },
  {
    "prompts": "Rostelecom’un fiber internet hızları genellikle tatmin edici, fakat bazı kullanıcılar bağlantı kesintilerinden şikayetçi.",
    "references": {
      "detect": ["Rostelecom’un fiber internet hızları", "bağlantı kesintileri"],
      "sentimentKeywords": {
        "Rostelecom’un fiber internet hızları": ["genellikle tatmin edici"],
        "bağlantı kesintileri": ["bazı kullanıcılar şikayetçi"]
      }
    }
  },
  {
    "prompts": "A1 Telekom’un mobil uygulama performansı genellikle iyi, ancak bazı kullanıcılar uygulamanın sık sık çökmesinden şikayet ediyor.",
    "references": {
      "detect": ["A1 Telekom’un mobil uygulama performansı", "uygulama çökmesi"],
      "sentimentKeywords": {
        "A1 Telekom’un mobil uygulama performansı": ["genellikle iyi"],
        "uygulama çökmesi": ["bazı kullanıcılar sık sık yaşadığını şikayet ediyor"]
      }
    }
  },
  {
    "prompts": "Eircom’un internet hizmetleri genellikle hızlı, fakat bazı kullanıcılar müşteri desteği konusunda yetersizlik olduğunu belirtiyor.",
    "references": {
      "detect": ["Eircom’un internet hizmetleri", "müşteri desteği"],
      "sentimentKeywords": {
        "Eircom’un internet hizmetleri": ["genellikle hızlı"],
        "müşteri desteği": ["bazı kullanıcılar yetersizlik olduğunu belirtiyor"]
      }
    }
  }


]



train = pd.DataFrame(data)


# train, test = train_test_split(df, train_size = 0.75, shuffle=True, random_state = 42)
train.shape

In [ ]:
train.head()

In [ ]:
dataset = Dataset.from_pandas(train)
dataset.column_names

In [ ]:
# Training Arguments
# Hyperparameters should beadjusted based on the hardware you using
training_arguments = TrainingArguments(
    output_dir= "trendyol_fine_tune",
    num_train_epochs= 1,
    per_device_train_batch_size= 16,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 5000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.01,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="none",
    push_to_hub=True
)

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="prompts",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub(commit_message="OK!")

In [ ]:
# Save the fine-tuned model
model_name = "trendyol_LLM_fine_tune"
trainer.model.save_pretrained(model_name)
model.config.use_cache = True
model.eval()

In [ ]:
# del model, trainer
# Clear the memory footprint
torch.cuda.empty_cache()
base_model =  "Trendyol/Trendyol-LLM-7b-chat-dpo-v1.0"
# Reload the base model
 
# LLM model ID
llm_model_id = "Trendyol/Trendyol-LLM-7b-chat-dpo-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model = PeftModel.from_pretrained(llm_model, "moarslan/trendyol_fine_tune")
model = model.merge_and_unload()

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
# #push the model to hub
# # !huggingface-cli login
# model.push_to_hub(model_name, use_temp_dir=False)
# tokenizer.push_to_hub(model_name, use_temp_dir=False)

In [15]:
aspect_extractor = ATEPC.AspectExtractor('multilingual',
                                         auto_device=True,  # False means load model on CPU
                                         cal_perplexity=True,
                                         )

# instance inference
aspect_extractor.predict(['I love this movie, it is so great!'],
                         save_result=True,
                         print_result=True,  # print the result
                         ignore_error=True,  # ignore the error when the model cannot predict the input
                         )

[2024-08-07 20:59:07] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-07 20:59:07] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-07 20:59:07] (2.4.1.post1) Downloading checkpoint:multilingual 
[2024-08-07 20:59:07] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-08-07 20:59:07] (2.4.1.post1) Checkpoint already downloaded, skip
[2024-08-07 20:59:07] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT
[2024-08-07 20:59:07] (2.4.1.post1) config: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.config
[2024-08-07 20:59:07] (2.4.1.post1) state_dict: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.state_dict
[2024-08-07 20:59:07] (2.4.1.post1) model: None
[2024-08-07 20:59:07] (2.4.1.post1) tokenizer: checkpoints/ATEPC

[{'sentence': 'I love this movie , it is so great !',
  'IOB': ['O', 'O', 'O', 'B-ASP', 'O', 'O', 'O', 'O', 'O', 'O'],
  'tokens': ['I',
   'love',
   'this',
   'movie',
   ',',
   'it',
   'is',
   'so',
   'great',
   '!'],
  'aspect': ['movie'],
  'position': [[3]],
  'sentiment': ['Positive'],
  'probs': [[0.0009741312242113054, 0.000678066280670464, 0.9983478784561157]],
  'confidence': [0.9983]}]

In [7]:
from pyabsa import AspectTermExtraction as ATEPC, available_checkpoints

# Multilingual model kontrol noktası yükleniyor
aspect_extractor = ATEPC.AspectExtractor('multilingual', auto_device=True)

# Metinler üzerinde aspect ve duygu analizi yapılması
text = ["Kitap 120 sayfa ve 40 cm"]

# Model çıktısı alınıyor
results = aspect_extractor.predict(text, save_result=True, print_result=False, ignore_error=True)

# Manuel aspect ekleme ve sentiment tahmini
manual_aspects = ["Kıck_turkey"]
for result in results:
    for aspect in manual_aspects:
        result['aspect'].append(aspect)
        
        # Sentiment tahmini için metni ve aspecti yeniden modelden geçirin
        sentiment_result = aspect_extractor.predict([text[0]], aspect_terms=[aspect], print_result=False, ignore_error=True)
        
        # Tahmin edilen sentimenti ekleyin
        if sentiment_result:
            sentiment_info = sentiment_result[0]['sentiment'][0]  # İlk sonucu alıyoruz
            result['sentiment'].append(sentiment_info)

# Sonuçları yazdırma
for result in results:
    print(f"Metin: {result['sentence']}")
    print(f"Aspectler: {result['aspect']}")
    print(f"Sentimentler: {result['sentiment']}")


[2024-08-08 12:12:58] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-08 12:12:58] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-08 12:12:58] (2.4.1.post1) Downloading checkpoint:multilingual 
[2024-08-08 12:12:58] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-08-08 12:12:58] (2.4.1.post1) Checkpoint already downloaded, skip
[2024-08-08 12:12:58] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT
[2024-08-08 12:12:58] (2.4.1.post1) config: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.config
[2024-08-08 12:12:58] (2.4.1.post1) state_dict: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.state_dict
[2024-08-08 12:12:58] (2.4.1.post1) model: None
[2024-08-08 12:12:58] (2.4.1.post1) tokenizer: checkpoints/ATEPC

IndexError: list index out of range

In [14]:
from pyabsa import AspectTermExtraction as ATEPC, available_checkpoints

# Load a multilingual model checkpoint
aspect_extractor = ATEPC.AspectExtractor('multilingual', auto_device=True)

# Predicting aspects and sentiments
text = ["Kitap 120 sayfa ve 40 cm "]
aspect_extractor.predict(text, save_result=True, print_result=True, ignore_error=True,aspect_terms=["Kıck_turkey"],)


[2024-08-07 20:58:01] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-07 20:58:01] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-07 20:58:01] (2.4.1.post1) Downloading checkpoint:multilingual 
[2024-08-07 20:58:01] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-08-07 20:58:01] (2.4.1.post1) Checkpoint already downloaded, skip
[2024-08-07 20:58:01] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT
[2024-08-07 20:58:01] (2.4.1.post1) config: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.config
[2024-08-07 20:58:01] (2.4.1.post1) state_dict: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.state_dict
[2024-08-07 20:58:01] (2.4.1.post1) model: None
[2024-08-07 20:58:01] (2.4.1.post1) tokenizer: checkpoints/ATEPC

[{'sentence': 'Kitap 120 sayfa ve 40 cm',
  'IOB': ['O', 'O', 'O', 'O', 'O', 'O'],
  'tokens': ['Kitap', '120', 'sayfa', 've', '40', 'cm'],
  'aspect': [],
  'position': [],
  'sentiment': [],
  'probs': [],
  'confidence': []}]

In [ ]:
# Predicting aspects and sentiments
text = ["Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz.  Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? @Turkcell"]
aspect_extractor.predict(text, save_result=True, print_result=True, ignore_error=True)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Model ve tokenizer'ı yükleyin
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Örnek bir metin
text = "Bu dizüstü bilgisayarın batarya ömrü mükemmel ancak klavyesi berbat."

# Metni tokenize edin
inputs = tokenizer(text, return_tensors="pt")

# Modeli kullanarak tahmin yapın
outputs = model(**inputs)
logits = outputs.logits

print(logits)


In [ ]:
from pyabsa import ABSADatasetList, SentimentClassifier, available_checkpoints

# Kullanılabilir kontrol noktalarını kontrol edin
available_checkpoints()

# Modeli ve veri kümesini seçin (Türkçe için uygun bir model ve veri kümesi olmalı)
model_path = 'checkpoints/bert_spc_acc_89.42_f1_79.07'
dataset_path = ABSADatasetList.Laptop14  # Uygun bir Türkçe veri kümesi bulmalısınız

# Modeli yükleyin
classifier = SentimentClassifier(model_path=model_path, dataset_path=dataset_path)

# Bir örnek metin üzerinde tahmin yapın
text = "Bu dizüstü bilgisayarın batarya ömrü mükemmel ancak klavyesi berbat."
aspect = "batarya ömrü"

# Duygu analizini gerçekleştirin
result = classifier.infer(text, aspect)

print(result)


In [ ]:
pyabsa.available_checkpoints()

In [5]:
from pyabsa import AspectTermExtraction as ATEPC

# Modeli yükle
aspect_extractor = ATEPC.AspectExtractor('multilingual', auto_device=True)

# Metin üzerinde sadece sentiment tahmini yapmak için modelin özelleştirilmiş kısmını kullanabilirsiniz
text = ["Kick_turkey'in kalitesi biraz zayıf. vodafone'nin daha da normal"]

# Manuel olarak belirlenmiş aspectler
manual_aspects = [
    ("Kick_turkey'in kalitesi", ["kalitesi"]),
    ("vodafone'nin", ["rezil"])
]

# Aspect ve sentimentleri eşleştirme
for aspect, keywords in manual_aspects:
    print(f"Aspect: {aspect}")
    for keyword in keywords:
        sentiment_result = aspect_extractor.predict(
            text,
            aspect_terms=[keyword],
            save_result=False,
            print_result=False,
            ignore_error=True
        )
        # Sentiment analizini burada işleyebilirsiniz
        print(f"Keyword: {keyword}")
        print("Sentiment Results:")
        for result in sentiment_result:
            print(result)


[2024-08-08 12:09:44] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-08 12:09:44] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-08 12:09:44] (2.4.1.post1) Downloading checkpoint:multilingual 
[2024-08-08 12:09:44] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-08-08 12:09:44] (2.4.1.post1) Checkpoint already downloaded, skip
[2024-08-08 12:09:44] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT
[2024-08-08 12:09:44] (2.4.1.post1) config: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.config
[2024-08-08 12:09:44] (2.4.1.post1) state_dict: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.state_dict
[2024-08-08 12:09:44] (2.4.1.post1) model: None
[2024-08-08 12:09:44] (2.4.1.post1) tokenizer: checkpoints/ATEPC

In [6]:
from pyabsa import AspectTermExtraction as ATEPC, available_checkpoints

# you can view all available checkpoints by calling available_checkpoints()
checkpoint_map = available_checkpoints()

aspect_extractor = ATEPC.AspectExtractor('multilingual',
                                         auto_device=True,  # False means load model on CPU
                                         cal_perplexity=True,
                                         )

# instance inference
aspect_extractor.predict(['I love this movie, it is so great!'],
                         save_result=True,
                         print_result=True,  # print the result
                         ignore_error=True,  # ignore the error when the model cannot predict the input
                         )

inference_source = ATEPC.ATEPCDatasetList.Restaurant16
atepc_result = aspect_extractor.batch_predict(target_file=inference_source,  #
                                              save_result=True,
                                              print_result=True,  # print the result
                                              pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                                              )

print(atepc_result)



[2024-08-07 20:29:31] (2.4.1.post1) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2024-08-07 20:29:31] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-07 20:29:31] (2.4.1.post1) ********** Available ATEPC model checkpoints for Version:2.4.1.post1 (this version) **********
[2024-08-07 20:29:31] (2.4.1.post1) Downloading checkpoint:multilingual 
[2024-08-07 20:29:31] (2.4.1.post1) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2024-08-07 20:29:31] (2.4.1.post1) Checkpoint already downloaded, skip
[2024-08-07 20:29:31] (2.4.1.post1) Load aspect extractor from checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT
[2024-08-07 20:29:31] (2.4.1.post1) config: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.config
[2024-08-07 20:29:31] (2.4.1.post1) state_dict: checkpoints/ATEPC_MULTILINGUAL_CHECKPOINT/fast_lcf_atepc.state_dict
[2024-0

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[2024-08-07 20:29:50] (2.4.1.post1) The results of aspect term extraction have been saved in /kaggle/working/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2024-08-07 20:29:50] (2.4.1.post1) Example 0: I love this <movie:Positive Confidence:0.9983> , it is so great !
[2024-08-07 20:29:50] (2.4.1.post1) Try to download 116.Restaurant16 dataset from https://github.com/yangheng95/ABSADatasets
[2024-08-07 20:29:50] (2.4.1.post1) Clone ABSADatasets from https://github.com/yangheng95/ABSADatasets.git
[2024-08-07 20:29:56] (2.4.1.post1) loading: integrated_datasets/apc_datasets/110.SemEval/116.restaurant16/restaurant_test.raw.inference


classifying aspect sentiments: 100%|██████████| 23/23 [00:06<00:00,  3.46it/s]

[2024-08-07 20:30:11] (2.4.1.post1) The results of aspect term extraction have been saved in /kaggle/working/Aspect Term Extraction and Polarity Classification.FAST_LCF_ATEPC.result.json
[2024-08-07 20:30:11] (2.4.1.post1) Example 0: serves really good <sushi:Positive Confidence:0.9861> .
[2024-08-07 20:30:11] (2.4.1.post1) Example 1: not the biggest <portions:Neutral Confidence:0.6267> but adequate .
[2024-08-07 20:30:11] (2.4.1.post1) Example 2: green <tea:Positive Confidence:0.993> creme <brulee:Positive Confidence:0.9974> is a must !
[2024-08-07 20:30:11] (2.4.1.post1) Example 3:   – i ca n ' t say enough about <this:Positive Confidence:0.9984> place .
[2024-08-07 20:30:11] (2.4.1.post1) Example 4: it has great <sushi:Positive Confidence:0.9975> and even better <service:Positive Confidence:0.9973> .
[2024-08-07 20:30:11] (2.4.1.post1) Example 5: the entire <staff:Positive Confidence:0.9973> was extremely accomodating and tended to my every need .
[2024-08-07 20:30:11] (2.4.1.post1)